## We will be focusing on using lightgbm model
What we will be doing in this notebook
1. 5-fold cross validation with optuna for hyperparameter tuning
2. n_model ensemble prediction

In [91]:
#sklearn libraries
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

#models
from lightgbm import LGBMRegressor

#pandas
import pandas as pd
import numpy as np

#optimization
import optuna

#ploting lib
import plotly.express as px

#python libraries
import pickle
import json

In [71]:
class CONFIG:
    numeric_features= ["max_floor_lvl","floor_area_sqm","sale_year","remaining_lease_months","latitude","longitude"]
    categorical_features = ['town','flat_type','flat_model','storey_range','residential',
                            'commercial','market_hawker','miscellaneous','multistorey_carpark',
                            'precinct_pavilion',"sale_month"]
    n_splits = 10

## Data preparation

In [82]:
preprocessed_data = pd.read_csv("../data/processed_df.csv", index_col=0)
preprocessed_data= preprocessed_data.drop_duplicates()
preprocessed_data = preprocessed_data.sample(frac=1)
preprocessed_data = preprocessed_data.reset_index(drop=True)
preprocessed_data.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,resale_price,max_floor_lvl,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,latitude,longitude,sale_month,sale_year,remaining_lease_months
0,SENGKANG,4 ROOM,13 TO 15,95.0,Model A,475000.0,17.0,Y,N,N,Y,N,N,1.382595,103.902573,1,2013,1032
1,QUEENSTOWN,3 ROOM,01 TO 03,67.0,Improved,358000.0,10.0,Y,N,N,Y,N,N,1.306485,103.796500,8,2021,621
2,CENTRAL AREA,3 ROOM,07 TO 09,60.0,Improved,425000.0,21.0,Y,N,N,N,N,N,1.298768,103.852176,8,2016,732
3,JURONG EAST,4 ROOM,04 TO 06,101.0,Model A,428000.0,10.0,Y,N,N,N,N,N,1.341967,103.746945,7,2018,954
4,BEDOK,3 ROOM,04 TO 06,68.0,Improved,315000.0,15.0,Y,Y,N,N,N,N,1.323405,103.941168,6,2016,816


In [83]:
preprocessed_data

,town,flat_type,storey_range,floor_area_sqm,flat_model,resale_price,max_floor_lvl,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,latitude,longitude,sale_month,sale_year,remaining_lease_months
0,SENGKANG,4 ROOM,13 TO 15,95.0,Model A,475000.0,17.0,Y,N,N,Y,N,N,1.382595,103.902573,1,2013,1032
1,QUEENSTOWN,3 ROOM,01 TO 03,67.0,Improved,358000.0,10.0,Y,N,N,Y,N,N,1.306485,103.796500,8,2021,621
2,CENTRAL AREA,3 ROOM,07 TO 09,60.0,Improved,425000.0,21.0,Y,N,N,N,N,N,1.298768,103.852176,8,2016,732
3,JURONG EAST,4 ROOM,04 TO 06,101.0,Model A,428000.0,10.0,Y,N,N,N,N,N,1.341967,103.746945,7,2018,954
4,BEDOK,3 ROOM,04 TO 06,68.0,Improved,315000.0,15.0,Y,Y,N,N,N,N,1.323405,103.941168,6,2016,816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184162,WOODLANDS,3 ROOM,10 TO 12,88.0,New Generation,275000.0,14.0,Y,N,N,Y,N,N,1.443349,103.772944,3,2018,730
184163,YISHUN,5 ROOM,07 TO 09,121.0,Improved,498000.0,12.0,Y,N,N,N,N,N,1.431353,103.837696,1,2016,816
184164,BUKIT BATOK,EXECUTIVE,01 TO 03,148.0,Maisonette,580000.0,4.0,Y,N,N,N,N,N,1.361721,103.749848,7,2020,777
184165,SENGKANG,4 ROOM,13 TO 15,90.0,Model A2,440000.0,15.0,Y,N,N,Y,N,N,1.393844,103.897494,5,2018,980


In [84]:
features = preprocessed_data.drop("resale_price", axis=1)
target = preprocessed_data['resale_price']

## Model
We are going to wrap all the construction into optuna objective

In [55]:
def get_model(model_args):
    #numeric_transformer
    numeric_transformer = Pipeline(steps=[
        ("normalize", StandardScaler()),
        ("polyfeatures", PolynomialFeatures())
    ])

    #categorical_transformer
    categorical_transformer = Pipeline(steps=[
        ("onehotencoding", OneHotEncoder(drop='first'))
    ])


    # Column transformer
    preprocessing = ColumnTransformer(transformers=[
        ('numerical_columns', numeric_transformer, CONFIG.numeric_features),
        ("categorical_column", categorical_transformer, CONFIG.categorical_features)
    ])

    #model definition and optuna suggestions
    #following the documentation at https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html, we will
    #be tuning 3 important hyperparameters 1. num_leaves 2. min_data_in_leaf  3. max_depth
    
    model = LGBMRegressor(**model_args)

    # main pipeline
    full_model_pipeline = Pipeline(steps=[
        ("preprocessing", preprocessing),
        ("clf", model)
    ])
    
    return full_model_pipeline


In [58]:
def objective(trial):
    num_leaves = trial.suggest_int("num_leaves",19,25,step=2)
    n_estimators = trial.suggest_int("n_estimators",2000,6000, step=100)
#     min_data_in_leaf  = trial.suggest_int("min_data_in_leaf",100,2000, step=250)
#     max_depth = trial.suggest_int("max_depth",7,13,step=2)
    learning_rate = trial.suggest_loguniform("learning_rate", 0.01,0.3)
    
    model_args={'num_leaves': num_leaves, 
                'n_estimators': n_estimators, 
                'max_depth': -1, 
                'learning_rate': learning_rate,
                "metric" : "rmse"}
    
    full_model_pipeline = get_model(model_args)
    
    mape = -1.0* np.mean(cross_val_score(full_model_pipeline,features,target,cv=5,scoring='neg_root_mean_squared_error'))
    
    return mape

In [59]:
study=optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=35)

[I 2021-12-02 20:41:36,680] A new study created in memory with name: no-name-61c28aa1-cc70-4d6c-884c-5ef1f614dc5b
[I 2021-12-02 20:45:05,057] Trial 0 finished with value: 24722.108452252105 and parameters: {'num_leaves': 23, 'n_estimators': 3500, 'learning_rate': 0.016193027448381463}. Best is trial 0 with value: 24722.108452252105.
[I 2021-12-02 20:48:04,533] Trial 1 finished with value: 22421.625731660897 and parameters: {'num_leaves': 19, 'n_estimators': 4000, 'learning_rate': 0.13599971927365137}. Best is trial 1 with value: 22421.625731660897.
[I 2021-12-02 20:52:26,980] Trial 2 finished with value: 22373.38288908499 and parameters: {'num_leaves': 21, 'n_estimators': 5400, 'learning_rate': 0.055445061007302204}. Best is trial 2 with value: 22373.38288908499.
[I 2021-12-02 20:57:21,544] Trial 3 finished with value: 23439.20520299997 and parameters: {'num_leaves': 21, 'n_estimators': 5600, 'learning_rate': 0.020186195097758042}. Best is trial 2 with value: 22373.38288908499.
[I 2021

In [60]:
print(study.best_trial)

FrozenTrial(number=33, values=[22270.37739466198], datetime_start=datetime.datetime(2021, 12, 2, 22, 53, 41, 864093), datetime_complete=datetime.datetime(2021, 12, 2, 22, 57, 57, 44626), params={'num_leaves': 23, 'n_estimators': 5400, 'learning_rate': 0.09069317714796007}, distributions={'num_leaves': IntUniformDistribution(high=25, low=19, step=2), 'n_estimators': IntUniformDistribution(high=6000, low=2000, step=100), 'learning_rate': LogUniformDistribution(high=0.3, low=0.01)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=33, state=TrialState.COMPLETE, value=None)


## Ensembling model based on best params
{'num_leaves': 23, 'n_estimators': 5400, 'learning_rate': 0.09069317714796007}

### Testing the performance of 1 model

In [61]:
best_params = {'num_leaves': 23, 'n_estimators': 5400, 'learning_rate': 0.09069317714796007}
full_model_pipeline = get_model(best_params)

X_train, X_text, y_train, y_test = train_test_split(features, target, train_size=0.9, random_state=42)
full_model_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical_columns',
                                                  Pipeline(steps=[('normalize',
                                                                   StandardScaler()),
                                                                  ('polyfeatures',
                                                                   PolynomialFeatures())]),
                                                  ['max_floor_lvl',
                                                   'floor_area_sqm',
                                                   'sale_year',
                                                   'remaining_lease_months',
                                                   'latitude', 'longitude']),
                                                 ('categorical_column',
                                                  Pipeline(steps=[('onehotencoding',
                                         

In [62]:
y_pred = full_model_pipeline.predict(X_text)

In [64]:
px.scatter(x=y_pred,y=y_test,title="Actual against Prediction")

## Ensembling 10 models with 10 fold cv
As we can see from above, the housing cost is not evenly distributed and therefore, we are going create a stratifiedKfold for continuous variable

In [99]:
class ContinuousStratifiedKFold(StratifiedKFold):
    #using parent init
    def split(self,X,y,groups=None):
        #using the same signature as original
        n_bins = 8
        bins = pd.cut(y, bins=n_bins, labels=False)
        return super().split(X,bins,groups=groups)

In [100]:
fold_splitter = ContinuousStratifiedKFold(n_splits=CONFIG.n_splits,shuffle=True, random_state=42)

result_log = {}
y_test_log = []
y_pred_log = []

for n_fold, (train_idx,test_idx) in enumerate(fold_splitter.split(features, target)):
    print(f"Training fold:{n_fold}")
    X_train=features.iloc[train_idx]
    y_train = target.iloc[train_idx]
    
    X_test = features.iloc[test_idx]
    y_test = target.iloc[test_idx]
    y_test_log.append(y_test)
    
    model_pipeline = get_model(best_params)
    model_pipeline.fit(X_train,y_train)
    y_pred = model_pipeline.predict(X_test)
    y_pred_log.append(y_pred)

    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test,y_pred)
    
    result_dict = dict(rmse=rmse, mae=mae, mape=mape)
    print(result_dict)
    result_log[n_fold] = result_dict
    
    #saving model
    model_dir = f"../artifacts/model_pipeline_fold_{n_fold}.pkl"
    with open(model_dir, 'wb') as pklfile:
        pickle.dump(model_pipeline, pklfile, protocol=pickle.HIGHEST_PROTOCOL)
        print("Model saved")
     

Training fold:0
{'rmse': 22056.706245250327, 'mae': 15924.466850176379, 'mape': 0.03624192949624733}
Model saved
Training fold:1
{'rmse': 21831.137152816664, 'mae': 15839.181135665893, 'mape': 0.03597396064225636}
Model saved
Training fold:2
{'rmse': 21901.69226723495, 'mae': 15844.617766010837, 'mape': 0.03601755374828156}
Model saved
Training fold:3
{'rmse': 22109.457708187096, 'mae': 16045.02300710227, 'mape': 0.03642136873567148}
Model saved
Training fold:4
{'rmse': 22291.40688218235, 'mae': 16077.142763416807, 'mape': 0.036554949785866424}
Model saved
Training fold:5
{'rmse': 22535.31566354448, 'mae': 16085.409047851272, 'mape': 0.036364743323599284}
Model saved
Training fold:6
{'rmse': 22097.103341971397, 'mae': 16003.715874020243, 'mape': 0.03629088747634874}
Model saved
Training fold:7
{'rmse': 22100.2137036957, 'mae': 16006.388303144073, 'mape': 0.036239992117598326}
Model saved
Training fold:8
{'rmse': 22072.112464300117, 'mae': 15938.281357533724, 'mape': 0.03613136359906501

In [101]:
## saving results
with open("../artifacts/metrics_result.json", 'w') as jsonfile:
    json.dump(result_log, jsonfile)